In [1]:
import sys
import os
# Add the directory containing 'package_folder' to the Python path
sys.path.append(os.path.abspath('..'))


In [2]:
# The package_folder is a package, so we don't need to import it directly.
# Instead, we will import the specific modules we need.
from package_folder.scaling_pipeline import transform_user_inputs  # Make sure these are implemented
from package_folder.weighted_sum import weighted_sum  # Make sure these are implemented

In [3]:
#real example of user inputs passed fron the frontend
user_inputs = {
"climate_preference":"hot",
"climate_importance":2,
"cost_of_living_importance":3,
"max_monthly_budget":2000,
"healthcare_importance":4,
"safety_importance":5,
"internet_speed_importance":6}

In [ ]:
import pickle
# After loading the pipeline
with open('../models/scaling_pipeline.pkl', 'rb') as f:
    pipe = pickle.load(f)
        
    # Inspect pipeline components
print("Pipeline steps:", pipe.named_steps)
    # If it's a ColumnTransformer, check the columns it expects
if hasattr(pipe, 'transformers_'):
    for name, transformer, columns in pipe.transformers_:
        print(f"Transformer '{name}' applied to columns: {columns}")

In [ ]:
processed_inputs = transform_user_inputs(user_inputs)
processed_inputs

In [8]:
import pandas as pd
data = pd.read_csv("../raw_data/merged_country_level/scaled_merged_data_after_imputation.csv")
result_df = weighted_sum(data, processed_inputs)


In [ ]:
result_df

In [ ]:
def recommend_countries(user_inputs):
    
    # Step 1: Convert user inputs to numerical weights
    processed_inputs = transform_user_inputs(user_inputs)

    # Step 2: Score countries based on user preferences
    data = pd.read_csv("../raw_data/merged_country_level/scaled_merged_data_after_imputation.csv")
    result_df = weighted_sum(data, processed_inputs)

    # Step 3: Return top 5 countries (currently the weighted sum is already returning the top 5 countries sorted)
    #top_5 = result_df.sort_values(by="country_user_score", ascending=False).head(5)

    return result_df.to_dict(orient="records")

recommend_countries(user_inputs)

In [2]:
import pickle

# Load the pipeline
with open('../models/scaling_pipeline.pkl', 'rb') as file:
    pipeline = pickle.load(file)

# Check the expected feature names
if hasattr(pipeline, 'named_steps'):
    # If the pipeline has named steps, you can check the last step
    last_step = list(pipeline.named_steps.values())[-1]
    if hasattr(last_step, 'feature_names_in_'):
        print("Expected feature names:", last_step.feature_names_in_)
    else:
        print("The last step does not have feature names.")
else:
    print("The pipeline does not have named steps.")

Expected feature names: ['average_monthly_cost_$' 'average_yearly_temperature'
 'internet_speed_mbps' 'safety_index' 'Healthcare Index']


In [3]:
import pickle

# Load the pipeline
with open('../models/v2_scaler_pipeline.pkl', 'rb') as file:
    pipeline = pickle.load(file)

# Check the expected feature names
if hasattr(pipeline, 'named_steps'):
    # If the pipeline has named steps, you can check the last step
    last_step = list(pipeline.named_steps.values())[-1]
    if hasattr(last_step, 'feature_names_in_'):
        print("Expected feature names:", last_step.feature_names_in_)
    else:
        print("The last step does not have feature names.")
else:
    print("The pipeline does not have named steps.")

Expected feature names: ['Unnamed: 0' 'average_monthly_cost_$' 'average_yearly_temperature'
 'internet_speed_mbps' 'safety_index' 'Healthcare Index']


In [5]:
import pickle
import pandas as pd

# Load the pipeline
with open('../models/v2_scaler_pipeline.pkl', 'rb') as file:
    pipeline = pickle.load(file)

def test_scaling(budget_input):
    # Get the expected feature names from the column transformer
    pipeline_features = pipeline.named_steps['column_transformer'].get_feature_names_out()

    # Create a DataFrame with all expected features, setting all to 0 except for the budget
    budget_df = pd.DataFrame([{ 
        feat: budget_input if feat == 'average_monthly_cost_$' else 0 
        for feat in pipeline_features
    }], columns=pipeline_features)

    # Use the specific column transformer for consistency
    column_transformer = pipeline.named_steps['column_transformer']
    scaled_budget = column_transformer.transform(budget_df)[0][0]  # Budget is the first column

    print(f"Input Budget: {budget_input}, Scaled Budget: {scaled_budget}")

# Test with a sample budget
test_scaling(1000)  # Replace 1000 with any budget value you want to test

KeyError: 'column_transformer'